## Введение в обработку естественного языка
### Урок 2. Создание признакового пространства
### Практическое задание

**Продолжим обработку данных с Твиттера.** 

In [1]:
from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:77% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import time

In [3]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [4]:
# Загрузка из файла
combine_df = pd.read_pickle('../lesson01/combine_df.pkl')
# Сохранение в файл
# combine_df.to_pickle('../lesson01/combine_df.pkl')

In [5]:
combine_df.head()

,id,label,tweet,clean_tweet,clean_tweet_token,clean_tweet_token_filtered,clean_tweet_stemmed,clean_tweet_lemmatized
0,1,0.00000,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]","[father, dysfunctional, selfish, drag, kid, dysfunction, run]"
1,2,0.00000,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]","[thank, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]"
2,3,0.00000,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.00000,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.00000,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


**1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**  
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df</li>
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000</li>
<li>Исключим стоп-слова с помощью stop_words='english'</li>
<li>Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names()</li>

**2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df</li>
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000</li>
<li>Исключим стоп-слова с помощью stop_words='english'</li>
<li>Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names()</li>

**3. Создайте мешок слов с помощью sklearn.feature_extraction.text.HashingVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df</li>
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.(можно изменить)</li>
<li>Исключим стоп-слова с помощью stop_words='english'</li>

**4. Проверьте ваши векторайзеры на корпусе который использовали на вебинаре, составьте таблицу метод векторизации и скор который вы получили (в методах векторизации по изменяйте параметры что бы добиться лучшего скора) обратите внимание как падает/растёт скор при уменьшении количества фичей, и изменении параметров, так же попробуйте применить к векторайзерам PCA для сокращения размерности посмотрите на качество сделайте выводы**